<h2><B><left>Hello, This Notebook Contains Example of Corona Virus Tweets Multi Class Classification.</left></B></h2>
 <h4><left>- Classes is: Extremely Positive, Positive, Extremely Negative, Negative. </left></h4>
<h4><I><left>- We do tweets cleaning, tokenization, normalization, also get the binary classes.</left></I></h4>
<h4><I><left>- Try to generate tweets from data.</left></I></h4>
<h4><I><left>- Vectorizing texts usiing different vectorization way: BOW, TF-IDF and finally Word Embedding.</left></I></h4>
<h4><I><left>- Train different Models(Classifiers) from Scikit-Learn library such as: SVM, Logistic Regression and MulinomialNB for classification.</left></I></h4>
<h4><I><left>- Train Deep Neural Networks Models with Tensorflow and Keras such as: LSTM, BiLSTM, CNN and FC for both multi class classification and binary classification.</left></I></h4>

<h4><I><left>- Finally we compare results and save the best Model.</left></I></h4>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
            <li><h3>Contents:</h3></li>
   <ul>
        <li><h4><a href="#import">Import Libraries and Helper Functions.</a></h4></li>
        <li><h4><a href="#prep">Prepocessing.</a></h4></li>
        <li><h4><a href="#generate">Create Tweets Generator.</a></h4></li>
        <li><h4><a href="#machine">Modeling Using Machine-Learning Models.</a></h4></li>
         <ul>
            <li><a href="#bow_cls">classification with BOW vectors.</a></li>
            <li><a href="#tfidf_cls">classification with TF-IDF vectors.</a></li>
        </ul>
        <li><h4><a href="#deep_nn">Modeling Using Deep Neural Network Models.</a></h4></li>
         <ul>
            <li><a href="#feature_ext">feature extraction.</a></li>
            <li><a href="#multi_nn">multi class classification.</a></li>
            <li><a href="#binary_nn">binary class classification</a></li>
        </ul>  
       <li><h4><a href="#save">Save The Best Model.</a></h4></li>
    </ul>
    </ul>
</div>
<hr>

<a id="import"></a>
<h3><B><left>- Import Data and Helper Functions: </left></B></h3>

--------------

In [1]:
# import libs:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import random
import re 
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
from sklearn.preprocessing import LabelEncoder 
from nltk.lm import MLE
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split


In [6]:
def get_binary_labels(sentiment):
    if sentiment == "Positive":
        return 'pos'
    elif sentiment == "Extremely Positive":
        return 'pos'
    elif sentiment == "Extremely Negative":
        return 'neg'
    else:
        return 'neg'

In [7]:
stop_words = stopwords.words('english')
my_words = ['against','after','through','above','to','over',
           'few','no','nor','not','only','to','can','cant','will','just','dont','should','shouldnt'
           'now','couldnt','didnt','hasnt','havent','isnt','arent','should','shouldnt','need','neednt','was','wasnt','were',
            'werent']
final_stop_words = [word for word in stop_words if not word in my_words]

# Reading Dataset:{Two ways: binary labels and all labels} 
data_frame = pd.read_csv('E:\Books and Resources\Educational\Covid_19_tweets_train.csv',encoding='latin1')
labels = data_frame['Sentiment']
tweets = data_frame['OriginalTweet']


data_frame_binary = data_frame[data_frame.Sentiment != "Neutral"]
data_frame_binary["Sentiment"] = data_frame_binary["Sentiment"].apply(get_binary_labels)
binary_tweets = data_frame_binary['OriginalTweet']
binary_labels = data_frame_binary['Sentiment']

E:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [3]:
def result_cleaning(tweets):
    url_ptn = re.compile(r'http(s?)\S+')
    ptn = re.compile(r'[^A-Za-z _#@]')
    ptn2 = re.compile(r'(https\w+ ?)|(@\S+)|(#\S+)|# |@ |_')
    ptn3 = re.compile(r'([\w])\1+')
    new_tweets = []
    tweets = [tweet.lower() for tweet in tweets]
    
    tweets = [re.sub(url_ptn,'',tweet) for tweet in tweets]
    
    tweets = [re.sub(ptn,' ',tweet) for tweet in tweets]
    
    tweets = [re.sub(ptn2,'',tweet) for tweet in tweets]
    
    tweets_tokens = [[word for word in tweet.split() if not word in final_stop_words ] for tweet in tweets]
    for tweet in tweets_tokens:
        new_tweets.append(' '.join([word for word in tweet]))
        
    tweets = new_tweets
    
    tweets =[re.sub(ptn3,r'\1',tweet) for tweet in tweets]
    
    wnl = nltk.WordNetLemmatizer()
    tweets = [" ".join([wnl.lemmatize(word) for word in tweet.split() ])for tweet in tweets]

    return tweets

In [18]:
from keras.models import load_model
def testing_result(path):
    data_frame = pd.read_csv(path,encoding='latin1')
    
    #get data & labels
    tweets = data_frame['OriginalTweet']
    labels = data_frame['Sentiment']
    #cleaning
    cleaned_tweets = result_cleaning(tweets)

    # tokenization: 
    tweets_tokens = get_tokens(tweets)
    clean_tokens = get_tokens(cleaned_tweets)
    
    #one_hot_labels
    one_hot_en = OneHotEncoder(sparse=False)
    one_hot_labels = one_hot_en.fit_transform(labels.values.reshape(-1,1))
    
    #convert to sequences and padding:
    maxlen = 50
    X_test = tokenizer.texts_to_sequences(labels)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
    model = load_model("model_5_classes.h5")
    model.load_weights("model_5_classes_w.hdf5")
    
    loss, accuracy = model.evaluate(X_test, one_hot_labels, verbose=False)
    print(" accuracy on test is:  {:.4f}".format(accuracy))


In [5]:
def get_tokens(tweets):
    tweets_tokens = [[word for word in tweet.split() if not word in final_stop_words] for tweet in tweets]
    return tweets_tokens

In [5]:
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed):
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

<a id="prep"></a>
<h3><left>- Data Preprocessing: </left></h3>


------------------

In [8]:
# cleaning data:
cleaned_tweets = result_cleaning(tweets)
cleaned_binary_tweets = result_cleaning(binary_tweets)

# tokenization: 
tweets_tokens = get_tokens(tweets)
clean_tokens = get_tokens(cleaned_tweets)

# data splitting:
sentences_train_b, sentences_test_b, y_train_b, y_test_b = train_test_split(cleaned_binary_tweets, binary_labels, test_size=0.25, random_state=1000)
sentences_train, sentences_test, y_train, y_test = train_test_split(cleaned_tweets, labels, test_size=0.25, random_state=1000)

<a id="generate"></a>
<h3><left>- Tweets Generation: </left></h3>

-------------

In [19]:
# get Padded Sentence:
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, clean_tokens)

In [20]:
# creating Text Generation:
model_mle = MLE(n)
model_mle.fit(train_data, padded_sents)

generate_sent(model_mle,15,10)

'man home bakery instead supermarket not provided adequate guidance to consumer transportation warehousing agri market'

<a id="machine"></a>
<h3><B><left>- Modeling with Machine-Learning Classifiers:</left></B></h3>
<h4><I><left>- classification using BOW victors.</left></I></h4>
<h4><I><left>- classification using TF-IDF victors.</left></I></h4>

---------------

<a id="bow_cls"></a>
<h4><B><left>- classification using BOW victors:</left></B></h4>

In [21]:
# Create BOW:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from keras.wrappers.scikit_learn import KerasClassifier


vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(sentences_train)
vectorizer.transform(sentences_train).toarray()

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

Using TensorFlow backend.
E:\Applications\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Applications\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Applications\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Applications\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py

In [ ]:
# SVC Classifier: 
from sklearn.svm import SVC
classifier_svc = SVC()
classifier_svc.fit(X_train, y_train)

In [24]:
# SVC Score:
score = classifier_svc.score(X_test, y_test)
print('Accuracy for SVC: {:.4f}'.format(score))

Accuracy for SVC: 0.6104


In [25]:
# NuSVC Classifier:
from sklearn.svm import NuSVC
classifier_nu = NuSVC()
classifier_nu.fit(X_train, y_train)

NuSVC()

In [26]:
# NuSVC Score:
score = classifier_nu.score(X_test, y_test)
print('Accuracy for NuSVC: {:.4f}'.format(score))

Accuracy for NuSVC: 0.6357


<a id="tfidf_cls"></a>
<h4><B><left>- classification using TF-IDF victors:</left></B></h4>

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB ,BernoulliNB
from sklearn.svm import NuSVC,SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

# tfid vectorizer
tvec = TfidfVectorizer()

# classifiers
nuSVC = NuSVC()
mnb = MultinomialNB()
lr = LogisticRegression(solver = "lbfgs")
sgd = SGDClassifier()

# model piplines
model_nu = Pipeline([('vectorizer',tvec),('classifier',nuSVC)])
model_mnb = Pipeline([('vectorizer',tvec),('classifier',mnb)])
model_lr = Pipeline([('vectorizer',tvec),('classifier',lr)])
model_sgd = Pipeline([('vectorizer',tvec),('classifier',sgd)])

In [ ]:
#NuSvc Model:
model_nu.fit(sentences_train, y_train)
predictions_nu = model_nu.predict(sentences_test)
confusion_matrix(predictions_nu, y_test)

print("Accuracy Tf-Idf NuSVC: {:.4f}".format(accuracy_score(predictions_nu, y_test)))
print("Precision Tf-Idf NuSVC: {:.4f}".format(precision_score(predictions_nu, y_test, average = 'weighted')))
print("Recall Tf-Idf NuSVC: {:.4f}".format(recall_score(predictions_nu, y_test, average = 'weighted')))

In [30]:
#MNB Model:
model_mnb.fit(sentences_train, y_train)
predictions_mnb = model_mnb.predict(sentences_test)
confusion_matrix(predictions_mnb, y_test)

print("Accuracy Tf-Idf MNB : {:.4f}".format(accuracy_score(predictions_mnb, y_test)))
print("Precision Tf-Idf MNB: {:.4f}".format(precision_score(predictions_mnb, y_test, average = 'weighted')))
print("Recall Tf-Idf MNB: {:.4f}".format(recall_score(predictions_mnb, y_test, average = 'weighted')))

Accuracy Tf-Idf MNB : 0.3995
Precision Tf-Idf MNB: 0.7304
Recall Tf-Idf MNB: 0.3995


In [31]:
#LR Model:
model_lr.fit(sentences_train, y_train)
predictions_lr = model_lr.predict(sentences_test)
confusion_matrix(predictions_lr, y_test)

print("Accuracy Tf-Idf LR : {:.4f}".format(accuracy_score(predictions_lr, y_test)))
print("Precision Tf-Idf LR: {:.4f}".format( precision_score(predictions_lr, y_test, average = 'weighted')))
print("Recall Tf-Idf LR: {:.4f}".format(recall_score(predictions_lr, y_test, average = 'weighted')))

E:\Applications\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy Tf-Idf LR : 0.5944
Precision Tf-Idf LR: 0.6013
Recall Tf-Idf LR: 0.5944


In [32]:
#SGD Model:
model_sgd.fit(sentences_train, y_train)
predictions_sgd = model_sgd.predict(sentences_test)
confusion_matrix(predictions_sgd, y_test)

print("Accuracy Tf-Idf SGD : {:.4f}".format(accuracy_score(predictions_sgd, y_test)))
print("Precision Tf-Idf SGD: {:.4f}".format(precision_score(predictions_sgd, y_test, average = 'weighted')))
print("Recall Tf-Idf SGD: {:.4f}".format(recall_score(predictions_sgd, y_test, average = 'weighted')))

Accuracy Tf-Idf SGD : 0.5580
Precision Tf-Idf SGD: 0.6472
Recall Tf-Idf SGD: 0.5580


<a id="deep_nn"></a>
<h3><B><left>- Modeling using Deep Neural Networks:</left></B></h3>
<h4><I><left>- exteact sequences for feed into network.</left></I></h4>
<h4><I><left>- build Model for multi class classification with Embeddings Layer(to convert words into vectors).</left></I></h4>
<h4><I><left>- build Model for binary class classification with Embeddings Layer(to convert words into vectors).</left></I></h4>


---------------

<a id="feature_ext"></a>
<h4><B><left>- extract sequences for feed to network: </left></B></h4>


In [9]:
#Preprocessing

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

encoder = LabelEncoder()
labels_encoded = encoder.fit_transform(labels)

one_hot_en = OneHotEncoder(sparse=False)
one_hot_labels = one_hot_en.fit_transform(y_train.values.reshape(-1,1))
one_hot_labels_test = one_hot_en.fit_transform(y_test.values.reshape(-1,1))

one_hot_labels_b = one_hot_en.fit_transform(y_train_b.values.reshape(-1,1))
one_hot_labels_test_b = one_hot_en.fit_transform(y_test_b.values.reshape(-1,1))

# Tokenization:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(sentences_train)

tokenizer2 = Tokenizer(num_words=10000)
tokenizer2.fit_on_texts(sentences_train_b)


# Text to Sequence:
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

X_train_b = tokenizer2.texts_to_sequences(sentences_train_b)
X_test_b = tokenizer2.texts_to_sequences(sentences_test_b)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
vocab_size_b = len(tokenizer2.word_index) + 1  # Adding 1 because of reserved 0 index


# Padding:
maxlen = 50
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
                                         
X_train_b = pad_sequences(X_train_b, padding='post', maxlen=maxlen)
X_test_b = pad_sequences(X_test_b, padding='post', maxlen=maxlen)

Using TensorFlow backend.
E:\Applications\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Applications\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Applications\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Applications\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py

<a id="multi_nn"></a>
<h4><B><left>- build Model for multi class classification:</left></B></h4>

In [10]:
#NN for 5 Classes:
from keras.models import Sequential
from keras import layers
embedding_dim = 100
model_77 = Sequential()
model_77.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))


model_77.add(layers.Conv1D(128, 3, activation='relu'))

model_77.add(layers.Bidirectional(layers.LSTM(units=128,return_sequences=True)))
model_77.add(layers.GlobalMaxPooling1D())

model_77.add(layers.Dense(50, activation='tanh'))
model_77.add(layers.Dropout(0.3))
model_77.add(layers.Dense(20, activation='relu'))

model_77.add(layers.Dense(5,activation = 'softmax'))
model_77.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_77.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           2513600   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 128)           38528     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 256)           263168    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                12850     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)               

In [11]:
history = model_77.fit(X_train, one_hot_labels,
                    epochs=3,
                    verbose=1,
                    validation_split=0.2,
                    batch_size=128)


Train on 24693 samples, validate on 6174 samples
Epoch 1/3
24693/24693 [==============================] - 56s 2ms/step - loss: 1.2637 - accuracy: 0.4511 - val_loss: 0.8589 - val_accuracy: 0.6733
Epoch 2/3
24693/24693 [==============================] - 40s 2ms/step - loss: 0.6872 - accuracy: 0.7512 - val_loss: 0.6991 - val_accuracy: 0.7462
Epoch 3/3
24693/24693 [==============================] - 40s 2ms/step - loss: 0.5135 - accuracy: 0.8303 - val_loss: 0.6804 - val_accuracy: 0.7527


In [12]:
loss, accuracy = model_77.evaluate(X_train, one_hot_labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_77.evaluate(X_test, one_hot_labels_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.8573
Testing Accuracy:  0.7707


<a id="binary_nn"></a>
<h4><B><left>- build Model for binary class classification:</left></B></h4>

In [13]:
#NN for 2 Classes:
embedding_dim = 100
model_77_b = Sequential()
model_77_b.add(layers.Embedding(vocab_size_b, embedding_dim, input_length=maxlen))


model_77_b.add(layers.Conv1D(128, 3, activation='relu'))

model_77_b.add(layers.Bidirectional(layers.LSTM(units=128,return_sequences=True)))
model_77_b.add(layers.GlobalMaxPooling1D())

model_77_b.add(layers.Dense(50, activation='tanh'))
model_77_b.add(layers.Dropout(0.3))
model_77_b.add(layers.Dense(20, activation='relu'))

model_77_b.add(layers.Dense(2,activation = 'softmax'))
model_77_b.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_77_b.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           2299500   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 128)           38528     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 48, 256)           263168    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                12850     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)               

In [14]:
history = model_77_b.fit(X_train_b, one_hot_labels_b,
                    epochs=3,
                    verbose=1,
                    validation_split=0.2,
                    batch_size=128)

Train on 20066 samples, validate on 5017 samples
Epoch 1/3
20066/20066 [==============================] - 36s 2ms/step - loss: 0.5063 - accuracy: 0.7313 - val_loss: 0.3317 - val_accuracy: 0.8597
Epoch 2/3
20066/20066 [==============================] - 33s 2ms/step - loss: 0.2310 - accuracy: 0.9110 - val_loss: 0.2895 - val_accuracy: 0.8782
Epoch 3/3
20066/20066 [==============================] - 32s 2ms/step - loss: 0.1294 - accuracy: 0.9559 - val_loss: 0.3319 - val_accuracy: 0.8798


In [16]:
loss, accuracy = model_77_b.evaluate(X_train_b, one_hot_labels_b, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_77_b.evaluate(X_test_b, one_hot_labels_test_b, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.9614
Testing Accuracy:  0.8731


<a id="save"></a>
<h3><B><left>- Save the best model</left></B></h3>

In [17]:
# 5- Saving Models
model_77.save("model_5_classes.h5")
model_77.save_weights("model_5_classes_w.hdf5")

model_77_b.save("model_2_classes.h5")
model_77_b.save_weights("model_2_classes_w.hdf5")

<h3><I><center> ...The End...</center></I></h3>